In [0]:
from pyspark.sql.functions import *

try:
  # Configuration for accessing CosmosDB
  readConfig = {
    "spark.cosmos.accountEndpoint": "https://scc2122rjr.documents.azure.com:443/",
    "spark.cosmos.accountKey": "2Dyv0Dju0A8zQ9AfgmWvVy342SHoXuw2pdMGdbEosVnoFx3dXBvvmIEpyGOGoRSM4Hovql8XQpG6S7Z3wmOkIw==",
    "spark.cosmos.database": "scc2122db",
    "spark.cosmos.container": "Messages",
    "spark.cosmos.read.customQuery": "SELECT m.id, m.channel FROM Messages m"
  }

  # Connect via azure-cosmosdb-spark to create Spark DataFrame
  # Infer schema automaticaly
  msgs = spark.read.format("cosmos.oltp").options(**readConfig) \
                    .option("spark.cosmos.read.inferSchema.enabled", "true") \
                    .load()
  # Let's register the dataframe as a view
  msgs.createOrReplaceTempView("Messages")

    # Count the number of users registered in each channel
  result = spark.sql("""SELECT id , count(*) AS count FROM
                              (SELECT id FROM Messages)
                          GROUP BY id""")

  # Write configuration
  writeConfig = {
    "spark.cosmos.accountEndpoint": "https://scc2122rjr.documents.azure.com:443/",
    "spark.cosmos.accountKey": "2Dyv0Dju0A8zQ9AfgmWvVy342SHoXuw2pdMGdbEosVnoFx3dXBvvmIEpyGOGoRSM4Hovql8XQpG6S7Z3wmOkIw==",
    "spark.cosmos.database": "scc2122db",
    "spark.cosmos.container": "TrendingChannels"
  }

  # Write to Cosmos DB from the result DataFrame
  result.write.format("cosmos.oltp").options(**writeConfig) \
              .mode("append") \
              .save()
except Exception as e:
  print(e)